In [9]:
#import sys
#!{sys.executable} -m pip install pandas
#!{sys.executable} -m pip install matplotlib
#!{sys.executable} -m pip install scipy
#!{sys.executable} -m pip install mip
#!{sys.executable} -m pip install xlrd


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from pprint import pprint
from scipy import stats
import string
from mip import *

#pd.set_option('display.max_columns', None)
#pd.set_option('display.float_format', lambda x: '%.0f' % x)
##pd.set_option("display.float_format",lambda x: "{:,}".format(x).replace(",","'"))
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
#pd.set_option('display.float_format', lambda x: '%.0f' % x)
##pd.set_option("display.float_format",lambda x: "{:,}".format(x).replace(",","'"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

First we import the data from the Excel file into a Pandas dataframe. For convenience, we rename the columns as in the Excel file.

In [10]:
data = pd.read_excel('ORscheduling_Complete (1).xlsx')

letters = string.ascii_uppercase[:len(data.columns)]
rename = dict(zip(data.columns,letters))

data.rename(columns=rename,inplace=True)
#data.dropna(how='all',inplace=True)
data



,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R,S
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Data,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,# of Surgical Teams per Department per Day,NaN,NaN,NaN,NaN,NaN,NaN,Min # of ORs per day,NaN,NaN,NaN,NaN,NaN,Max # of ORs per day,NaN,NaN,NaN,NaN,NaN
4,NaN,M,T,W,R,F,NaN,M,T,W,R,F,NaN,M,T,W,R,F,NaN
5,Opthalmology,2,2,2,2,2,NaN,0,0,0,0,0,NaN,2,2,2,2,2,NaN
6,Gynecology,3,3,3,3,3,NaN,0,0,0,0,0,NaN,3,3,3,3,3,NaN
7,Oral Surgery,0,1,0,1,0,NaN,0,0,0,0,0,NaN,1,1,1,1,1,NaN
8,Otolaryngology,1,1,1,1,1,NaN,0,0,0,0,0,NaN,1,1,1,1,1,NaN
9,General Surgery,6,6,6,6,6,NaN,0,0,0,0,0,NaN,6,6,6,6,6,NaN


In [11]:
## Surgical Teams per Department per Day

print('Departments:\n')
teams = data.loc[5:9,'A'].values
print(teams)

print('\nDays:\n')
days = data.loc[4,'B':'F'].values
print(days)

print('\nTeams availability per department (rows) per day (columns):\n')
teams_available = data.loc[5:9,'B':'F'].values.astype(int)
print(teams_available)

## Min # of ORs per day
print('\nMinimum number of ORs per department (rows) per day (columns):\n')
min_ors = data.loc[5:9,'H':'L'].values.astype(int)
print(min_ors)

## Max # of ORs per day
print('\nMaximum number of ORs per department (rows) per day (columns):\n')
max_ors = data.loc[5:9,'N':'R'].values.astype(int)
print(max_ors)

## Weekly Department Requirements
print('\nWeekly department requirements (min/max):\n')
weekly_reqs = data.loc[13:17,'B':'C'].values.astype(int)
print(weekly_reqs)

print('\nWeekly targets per department:\n')
weekly_targets = data.loc[13:17,'H'].values.astype(float)
print(weekly_targets)


Departments:

['Opthalmology' 'Gynecology' 'Oral Surgery' 'Otolaryngology'
 'General Surgery']

Days:

['M' 'T' 'W' 'R' 'F']

Teams availability per department (rows) per day (columns):

[[2 2 2 2 2]
 [3 3 3 3 3]
 [0 1 0 1 0]
 [1 1 1 1 1]
 [6 6 6 6 6]]

Minimum number of ORs per department (rows) per day (columns):

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

Maximum number of ORs per department (rows) per day (columns):

[[2 2 2 2 2]
 [3 3 3 3 3]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [6 6 6 6 6]]

Weekly department requirements (min/max):

[[ 3  6]
 [12 18]
 [ 2  3]
 [ 2  4]
 [18 25]]

Weekly targets per department:

[ 39.4 117.4  19.9  26.3 189. ]


In [12]:


m = Model(name='Operation Rooms',sense=MAXIMIZE, solver_name=CBC) 

t = len(teams) ; d = len(days)

## Daily # ORs assigned is integer
x = [m.add_var(name='x'+str(i)+'_'+str(j),var_type=INTEGER, \
               lb = min_ors[i-1,j-1], ub= max_ors[i-1,j-1]) for i in range(1,t+1) for j in range(1,d+1) ]

#print(x)
m.objective = xsum(xsum(8/weekly_targets[i-1]*m.var_by_name('x'+str(i)+'_'+str(j)) \
                        for j in range(1,d+1)) for i in range(1,d+1)) 
## ----------------------------------------------------------------
## Total # ORs each day
## the total number of operating rooms each day has to be less than or equal to 10.
for j in range(1,d+1):
    m += xsum(m.var_by_name('x'+str(i)+'_'+str(j)) for i in range(1,t+1)) <=10 #, 'tot_per_day_'+str(j) 
## ----------------------------------------------------------------
## UB: Surgical Teams per day
for i in range(1,t+1):
    for j in range(1,d+1):
        m += m.var_by_name('x'+str(i)+'_'+str(j)) <= abs(teams_available[i-1,j-1]) #,'per_day_'+str(i)+'_'+str(j)

## ----------------------------------------------------------------        
## UB: Weekly Dept OR requirement
## LB: Weekly Dept OR requirement
for i in range(1,t+1):
    m += xsum(m.var_by_name('x'+str(i)+'_'+str(j)) for j in range(1,d+1)) >= weekly_reqs[i-1,0] #,'week_'+str(j)+' min'
for i in range(1,t+1):
    m += xsum(m.var_by_name('x'+str(i)+'_'+str(j)) for j in range(1,d+1)) <= weekly_reqs[i-1,1] #,'week_'+str(j)+' max'

    
## ----------------------------------------------------------------
## Departmental Targets
for i in range(1,t+1):
    m += 8*xsum(m.var_by_name('x'+str(i)+'_'+str(j)) for j in range(1,t+1)) <= weekly_targets[i-1] #,'weekly_target_'+str(i)


## ----------------------------------------------------------------        

m.write('model.lp')
with open('model.lp', 'r') as f:
    model = f.readlines()
for line in model:
    print(re.sub('\n','',line))
    
m.write('model.mps')
#with open('model.mps', 'r') as f:
#    model = f.readlines()
#for line in model:
#    print(line)


\Problem name: Operation Rooms

Minimize
OBJROW: -0.20305 x1_1 -0.20305 x1_2 -0.20305 x1_3 -0.20305 x1_4 -0.20305 x1_5 -0.06814 x2_1 -0.06814 x2_2 -0.06814 x2_3 -0.06814 x2_4 -0.06814 x2_5
 -0.40201 x3_1 -0.40201 x3_2 -0.40201 x3_3 -0.40201 x3_4 -0.40201 x3_5 -0.30418 x4_1 -0.30418 x4_2 -0.30418 x4_3 -0.30418 x4_4 -0.30418 x4_5
 -0.04233 x5_1 -0.04233 x5_2 -0.04233 x5_3 -0.04233 x5_4 -0.04233 x5_5
Subject To
constr(0):  x1_1 + x2_1 + x3_1 + x4_1 + x5_1 <= 10
constr(1):  x1_2 + x2_2 + x3_2 + x4_2 + x5_2 <= 10
constr(2):  x1_3 + x2_3 + x3_3 + x4_3 + x5_3 <= 10
constr(3):  x1_4 + x2_4 + x3_4 + x4_4 + x5_4 <= 10
constr(4):  x1_5 + x2_5 + x3_5 + x4_5 + x5_5 <= 10
constr(5):  x1_1 <= 2
constr(6):  x1_2 <= 2
constr(7):  x1_3 <= 2
constr(8):  x1_4 <= 2
constr(9):  x1_5 <= 2
constr(10):  x2_1 <= 3
constr(11):  x2_2 <= 3
constr(12):  x2_3 <= 3
constr(13):  x2_4 <= 3
constr(14):  x2_5 <= 3
constr(15):  x3_1 <= -0
constr(16):  x3_2 <= 1
constr(17):  x3_3 <= -0
constr(18):  x3_4 <= 1
constr(19):  x

In [13]:
SearchEmphasis = 1 ; LP_Method = 1 
m.max_gap = 0.05
m.preprocess = 1
status = m.optimize()


In [14]:
if status == OptimizationStatus.OPTIMAL:
    print('optimal solution cost {} found'.format(m.objective_value))
elif status == OptimizationStatus.FEASIBLE:
    print('sol.cost {} found, best possible: {}'.format(m.objective_value, m.objective_bound))
elif status == OptimizationStatus.NO_SOLUTION_FOUND:
    print('no feasible solution found, lower bound is: {}'.format(m.objective_bound))
elif status == OptimizationStatus.INFEASIBLE:
    print('no feasible solution found')
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print('solution:')
    for v in m.vars:
        if abs(v.x) > 1e-6: # only printing non-zeros
            print('{} : {}'.format(v.name, v.x))

optimal solution cost 4.456298750836373 found
solution:
x1_1 : 2.0
x1_3 : 2.0
x2_1 : 2.0
x2_2 : 3.0
x2_3 : 3.0
x2_4 : 3.0
x2_5 : 3.0
x3_2 : 1.0
x3_4 : 1.0
x4_3 : 1.0
x4_4 : 1.0
x4_5 : 1.0
x5_1 : 3.0
x5_2 : 6.0
x5_3 : 4.0
x5_4 : 5.0
x5_5 : 5.0
